In [1]:
import json
import pyperclip
import json
import re

# # 从文件读取 JSON
# with open("/Users/liuyiwei/python/tactful-tom/dataset/Tactful_conv_element_0.json", "r", encoding="utf-8") as f:
#     data = json.load(f)

# data = data[36]
# print(data["set_id"])
# # 1. 获取对话文本
# full_text = data["full_context"]

# # 2. 获取角色名称（是一个字典），并把它的所有 value（角色名）提取到列表
# characters_dict = data["characters"]
# speakers = list(characters_dict.values())  # ["Jamey", "Pearl", "Marithza", "Donyae"]

# 3. 定义一个函数，用于在角色名之前插入换行符
def bold_unbolded_speakers(text, speaker_list):
    for speaker in speaker_list:
        # 匹配不是加粗的 Name:，避免重复包裹 **
        # 要求前面不是 *，后面是冒号，但后面不能紧跟 **
        pattern = re.compile(rf'(?<!\*)\b({re.escape(speaker)}):(?!=\*)', re.IGNORECASE)
        text = pattern.sub(r'**\1:**', text)
    return text
def insert_line_breaks_bold_speakers(text, speaker_list):
    speaker_pattern = '|'.join(re.escape(s) for s in speaker_list)
    # 匹配所有的 **Speaker:** // **Speaker**:，无论前面是什么，直接加换行
    pattern = re.compile(rf'(\*\*(?:{speaker_pattern})(?::)?\*\*)')
    # 每个匹配前都加两个换行
    return pattern.sub(r'\n\n\1', text)

def remove_extra_blank_lines(text):
    # 把 3 个及以上连续换行缩减为 2 个
    return re.sub(r'\n{3,}', '\n\n', text)

# 定义一个正则，匹配常见的 emoji 区段
emoji_pattern = re.compile(
    "["  
    u"\U0001F600-\U0001F64F"  # Emoticons (表情)
    u"\U0001F300-\U0001F5FF"  # Symbols & Pictographs
    u"\U0001F680-\U0001F6FF"  # Transport & Map symbols
    u"\U0001F1E0-\U0001F1FF"  # Flags (iOS)
    "]+",
    flags=re.UNICODE
)

def remove_emojis(text: str) -> str:
    """
    从文本中移除所有匹配到的 emoji。
    """
    return emoji_pattern.sub(' ', text)

def remove_horizontal_lines(text):
    """
    Remove horizontal lines (three or more dashes) from the text.
    """
    # Match horizontal lines consisting of 3 or more dashes, optionally with whitespace
    horizontal_line_pattern = re.compile(r'\n\s*-{3,}\s*\n')
    return horizontal_line_pattern.sub('\n\n', text)

# Update the process_text function to include this new step
def process_text(text, speakers):
    fixed_text = remove_emojis(text)
    fixed_text = bold_unbolded_speakers(fixed_text, speakers)
    fixed_text = insert_line_breaks_bold_speakers(fixed_text, speakers)
    fixed_text = remove_horizontal_lines(fixed_text)  # Add this line
    fixed_text = remove_extra_blank_lines(fixed_text)
    return fixed_text.strip()

def sort_by_set_id(data):
    """
    Sort the data list by set_id in numerical order.
    set_id format is like "8-0-1" or "10-0-1" where each segment should be sorted numerically.
    """
    def set_id_key(item):
        # Split the set_id by '-' and convert each part to integer
        if "set_id" in item:
            try:
                parts = item["set_id"].split('-')
                # Convert each part to int for numerical sorting
                return tuple(int(p) for p in parts)
            except (ValueError, TypeError):
                # If conversion fails, return a tuple that will sort to the end
                return (float('inf'),) * 3
        return (float('inf'),) * 3  # Items without set_id go to the end
    
    # Sort the data using the custom key function
    return sorted(data, key=set_id_key)

# with open("/Users/liuyiwei/python/tactful-tom/dataset/Tactful_conv_element_0.json", "r", encoding="utf-8") as f:
#     data = json.load(f)

# speakers = list(characters_dict.values())  # e.g. ["Jamey", "Pearl", "Marithza", "Donyae"]

# fixed_text = remove_emojis(full_text)
# fixed_text = bold_unbolded_speakers(fixed_text, speakers)  # 👈 先加粗所有没加的说话人名
# fixed_text = insert_line_breaks_bold_speakers(fixed_text, speakers)  # 👈 然后插入换行
# fixed_text = remove_extra_blank_lines(fixed_text)
# fixed_text = fixed_text.strip()

# print(fixed_text)
# pyperclip.copy(fixed_text)

In [12]:
# 主流程：读取原 JSON 文件，遍历所有对话样本，然后对每个样本中的 full_context 进行格式化处理

# 输入与输出文件路径
input_filepath = "/Users/liuyiwei/python/tactful-tom/dataset/Tactful_conv_set_4.json"
output_filepath = "/Users/liuyiwei/python/tactful-tom/dataset/Tactful_conv_set_4.json"

with open(input_filepath, "r", encoding="utf-8") as fin:
    all_data = json.load(fin)

new_data = []
for sample in all_data:
    # 处理 full_context
    full_text = sample.get("full_context", "")
    characters_dict = sample.get("characters", {})
    speakers = list(characters_dict.values())
    sample["full_context"] = process_text(full_text, speakers)
    
    # 如果存在 short_context，也进行同样的处理
    short_text = sample.get("short_context", "")
    sample["short_context"] = process_text(short_text, speakers)

    new_data.append(sample)

# Sort the data
new_data = sort_by_set_id(new_data)

with open(output_filepath, "w", encoding="utf-8") as fout:
    json.dump(new_data, fout, ensure_ascii=False, indent=4)

print("处理完毕，新的 JSON 文件保存在：", output_filepath)

处理完毕，新的 JSON 文件保存在： /Users/liuyiwei/python/tactful-tom/dataset/Tactful_conv_set_4.json


In [18]:
# 打印一下查看是否修改好了格式
output_filepath = "/Users/liuyiwei/python/tactful-tom/dataset/Tactful_conv_set_0.json"
import json
from IPython.display import display, Markdown

with open(output_filepath, "r", encoding="utf-8") as f:
    data = json.load(f)

if data:
    sample = data[2]
    print("full_context:")
    print(sample["set_id"])
    # print(sample["full_context"])
    full_context = sample["full_context"]
    markdown_lines = []
    for line in full_context.split("\n"):
        if line.strip():  # 跳过空行
            markdown_lines.append(f"{line}  ")  # 两个空格是 Markdown 的换行标志

    markdown_text = "\n".join(markdown_lines)
    display(Markdown(markdown_text))

else:
    print("没有找到数据。")

full_context:
0-1-0-2


**Cornell:** So, we're all here at lunch, and I have to ask—what's everyone's favorite cuisine? I've been trying to expand my palate a bit.    
**Isobella:** That's a great question, Cornell! I think I'd have to go with Italian. There's just something so comforting about a bowl of pasta.    
**Arvilla:** Italian is solid, but for me, it's all about the spices with Indian cuisine. You just can't beat a good curry.    
**Mckenize:** Oooh, those are both good choices. I'm a fan of Japanese food, especially sushi. It's like art you can eat!    
**Cornell:** Those all sound amazing. I've been really into Mediterranean lately. There's so much variation with flavors and it's usually pretty healthy too.    
**Isobella:** Mediterranean is a great choice, Cornell! It’s hard not to love dishes that come with a side of warm pita bread.    
**Arvilla:** Agreed! And speaking of bread, Israeli cuisine has some of the best I've tasted. Shakshuka and hummus are my go-tos.    
**Mckenize:** Now I'm craving a little bit of everything! Maybe we should do a potluck lunch at some point and bring a dish from our favorite cuisine.    
**Cornell:** That's a fantastic idea, Mckenize. It would be a fun way to explore different cuisines without having to buy a plane ticket.    
**Isobella:** I'm on board! I'll whip up a pasta dish that'll knock your socks off. Let's plan it for next Friday!    
**Isobella:** Oh, I just remembered I need to quickly check something. Can you all excuse me for a moment? I want to confirm the details for an appointment I have coming up.    
**Mckenize:** And I need to step away to refill my water bottle. I'll be back soon.    
**Arvilla:** No problem! We’ll be right here when you return.     
**Cornell:** Sure thing. In the meantime, Arvilla, have you been to any of the newer restaurants around town recently?    
**Arvilla:** I have! There’s been a lot of buzz around a few of them, but some are a bit on the pricier side.    
**Cornell:** Yeah, I've noticed that too. Honestly, I usually prefer places that offer good food without the hefty price tag.     
**Arvilla:** I understand that completely. There's nothing better than finding a hidden gem with amazing food and a laid-back vibe.     
**Cornell:** Exactly! A cozy atmosphere and a friendly staff make the experience so much more enjoyable without emptying the wallet.    
**Arvilla:** For sure. There's this little diner on Elm Street that fits that description perfectly. Maybe we should go sometime!  
**Mckenize:** I'm back! Sorry, took longer than I expected. What's this about a new diner?    
**Cornell:** Hey, Mckenize! Arvilla was just telling me about a hidden gem on Elm Street. We were thinking of possibly checking it out.    
**Mckenize:** That sounds interesting! I'll have to try and join next time. So, any plans for this weekend?    
**Cornell:** Honestly, I'm still undecided. I might take a spontaneous road trip or just enjoy a relaxing weekend at home. How about you, Arvilla?    
**Arvilla:** I'm planning a hiking trip. The leaves are starting to change, and it's the perfect weather for it. You're both welcome to join if you're interested!    
**Mckenize:** That sounds wonderful! I'll think about it, but I just remembered I have to run some errands. I'll catch up with you all later.    
**Cornell:** Alright, Mckenize! Take care. Let us know if you're joining any of the plans.    
**Arvilla:** Bye, Mckenize! Take care, and good luck with the errands.  
**Isobella:** Hey everyone, I'm back! Thanks for waiting. So, speaking of fantastic meals, there's this new Italian place that just opened up in town. I was thinking it would be fun to try it out for dinner this weekend. Cornell, would you like to join me?    
**Cornell:** Welcome back, Isobella! That sounds like a lovely idea, but unfortunately, I already have something to do this weekend. Maybe another time.    
**Isobella:** Oh, no worries! I'll have to check it out and let you know how it is.    
**Arvilla:** If it's as good as I hear, I'm sure it'll be amazing. Let us know if it lives up to the hype!    
**Isobella:** Will do! And if it does, maybe we can all plan a visit sometime soon.    
**Cornell:** Sounds great! I hope you have a wonderful time, Isobella. Enjoy that Italian cuisine!  

In [12]:
# 打印一下查看是否修改好了格式
output_filepath = "/Users/liuyiwei/python/tactful-tom/dataset/final_elements/Tactful_conv_element_3.json"
import json
from IPython.display import display, Markdown

with open(output_filepath, "r", encoding="utf-8") as f:
    data = json.load(f)

if data:
    sample = data[0]
    print("full_context:")
    print(sample["set_id"])
    # print(sample["full_context"])
    full_context = sample["full_context"]
    markdown_lines = []
    for line in full_context.split("\n"):
        if line.strip():  # 跳过空行
            markdown_lines.append(f"{line}  ")  # 两个空格是 Markdown 的换行标志

    markdown_text = "\n".join(markdown_lines)
    display(Markdown(markdown_text))

else:
    print("没有找到数据。")

full_context:
3-11-1-2


**Darren:** So Crawford, I heard you have some exciting plans for the summer. What's this about learning piano?  
**Crawford:** Yeah, I'm starting lessons next week. I’ve always wanted to learn how to play some of those cool songs I hear in movies.  
**Shiv:** That sounds wonderful, Crawford! It's a great skill to have and so much fun. Have you decided on which songs you want to start with?  
**Crawford:** I’m thinking of starting with something simple, maybe “Chopsticks,” and then I'll work my way up to some movie scores.  
**Adrian:** Piano is such a fantastic instrument. You know, your mom was quite the pianist back in the day. She could play just about anything by ear!  
**Darren:** Oh yes, Shiv was really impressive. You should see if she’d be willing to play a duet with you, Crawford!   
**Shiv:** I'd love to. It would be great to play together. Maybe once you’re comfortable with the basics, we can try something fun as a duo.  
**Crawford:** That sounds awesome! It’ll be fun to play with you, Mom. Uncle Adrian, have you ever played an instrument?  
**Adrian:** I gave the guitar a go in college, but let's just say my talents lie elsewhere. I think I have more of a knack for listening than playing!  
**Darren:** Well, you’ll have a front-row seat to Crawford's piano recitals. I'm sure you’ll be our biggest fan!  
**Crawford:** Haha, thanks, Uncle Adrian! Speaking of music, I need to quickly troubleshoot my internet connection. I'll be right back!  
**Adrian:** And I need to pop out too. I'm having a moment wondering if I turned off the coffee maker. I'll catch you all later!  
[Crawford and Adrian leave the conversation.]  
**Darren:** Alright, it's just us now. You know, Shiv, I think Crawford could really excel with some encouragement, especially when it comes to sticking with music.  
**Shiv:** I agree. It can be disheartening to any budding musician when they're just beginning. Music theory is tough, but it’s not the only way to love music.  
**Darren:** You're right. Maybe we can focus on boosting his confidence by emphasizing the joy of making music rather than worrying too much about the technical details just yet.  
**Shiv:** Exactly. Plus, it could be a good idea to integrate some fun activities that relate to music theory, like composing his own simple tunes or experimenting with improvisation.   
**Darren:** I think a mix of guidance and letting him explore on his own might be the perfect balance. We can highlight his progress and remind him that it's okay to have challenges along the way.  
**Shiv:** Yes, and perhaps involving him in activities, like watching live performances or joining a local music club, can keep his interest alive and thriving.  
**Adrian:** Hey, I'm back! Coffee maker was safely off, false alarm. What did I miss?  
**Shiv:** Oh, welcome back, Adrian! We were just talking about how to keep Crawford motivated with his music lessons.  
**Adrian:** Ah, speaking of Crawford and music, wasn’t he trying out the violin before? I remember him mentioning something like that.  
**Darren:** Yes, he was taking some lessons. It’s a bit tricky for him, though. I think he finds the theory part overwhelming sometimes.  
**Shiv:** Right, and he mentioned that he sometimes struggles to discern notes accurately, which doesn't make it any easier. But there are plenty of ways to improve with practice.  
**Adrian:** Definitely. You know, I've read that sometimes just immersing oneself in the music can help naturally improve those skills over time. Anyway, I have to leave again. My phone's almost dead, and I need to charge it.  
**Darren:** Alright, Adrian! Thanks for popping back in. We'll catch up later.  
**Adrian:** Definitely. Good luck with the piano lessons, Crawford! Talk to you all soon.  
**Crawford:** Hey everyone, I'm back!  
**Darren:** Welcome back, Crawford! So, we were just chatting about your music lessons. Are you excited about them?  
**Crawford:** Definitely! I've been dreaming about being a musician, and I love the idea of creating my own music one day.  
**Shiv:** That's wonderful to hear! You’re really talented at music, and we truly believe you should keep that passion alive.  
**Darren:** Absolutely! Keep up that motivation, Crawford. Your love for music is inspiring.  
**Crawford:** Thanks, that means a lot to me. I can't wait to start learning and seeing where it takes me.  
**Shiv:** We’re always here cheering you on, Crawford. Make sure to enjoy the journey as much as the destination.  

#### 更新 lie_type

In [17]:
# 更新 lie_type

# 输入与输出文件路径
input_filepath = "/Users/liuyiwei/python/tactful-tom/dataset/Tactful_conv_set_3.json"
output_filepath = "/Users/liuyiwei/python/tactful-tom/dataset/Tactful_conv_set_3.json"

with open(input_filepath, "r", encoding="utf-8") as fin:
    all_data = json.load(fin)

new_data = []
for sample in all_data:
    
    # # 对于 Tactful_conv_element_0/4.json 文件中的所有样本
    # # 将 lie_type 设置为 "pareto_white_lies"
    # sample["lie_type"] = "pareto_white_lies"
    # sample["real_reason_type"] = False

    # 对于 Tactful_conv_element_1/2/3.json 文件中的所有样本
    # 将 lie_type 设置为 "altruistic_white_lies"
    sample["lie_type"] = "altruistic_white_lies"
    sample["real_reason_type"] = True

    if sample.get("truth_id") == 0:
        sample["truth_q"] = ""
        sample["truth_c"] = ""

    new_data.append(sample)


with open(output_filepath, "w", encoding="utf-8") as fout:
    json.dump(new_data, fout, ensure_ascii=False, indent=4)

#### 替换justificationoption和liability

In [37]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
将替换文件中的 `justificationQA` 与 `lieabilityQAs`
按 set_id 覆盖到目标文件中。
"""
import json
from pathlib import Path

# 文件路径
REPLACE_PATH = Path("/Users/liuyiwei/python/tactful-tom/dataset/替换justificationoption和liability/Tactful_conv_set_4.json")
TARGET_PATH  = Path("/Users/liuyiwei/python/tactful-tom/dataset/1. final_set/Tactful_conv_set_4.json")

# 是否先备份目标文件
MAKE_BACKUP = True

def load_json(path: Path):
    with path.open("r", encoding="utf-8") as f:
        return json.load(f)

def save_json(data, path: Path):
    with path.open("w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

def main():
    # 1. 读取两个文件
    replace_data = load_json(REPLACE_PATH)
    target_data  = load_json(TARGET_PATH)

    # 2. 构造 {set_id: {"justificationQA": ..., "lieabilityQAs": ...}}
    replace_lookup = {}
    for sample in replace_data:
        sid = sample.get("set_id")
        if not sid:
            continue
        replace_lookup[sid] = {
            "justificationQA": sample.get("justificationQA"),
            "lieabilityQAs":  sample.get("lieabilityQAs")
        }

    # 3. 备份
    if MAKE_BACKUP:
        bak = TARGET_PATH.with_suffix(TARGET_PATH.suffix + ".bak")
        bak.write_bytes(TARGET_PATH.read_bytes())
        print(f"已备份目标文件 -> {bak}")

    # 4. 替换
    num_replaced = 0
    for sample in target_data:
        sid = sample.get("set_id")
        repl = replace_lookup.get(sid)
        if repl:
            # 只有当替换文件中该字段存在时才更新，防止覆盖为 None
            if repl["justificationQA"] is not None:
                sample["justificationQA"] = repl["justificationQA"]
            if repl["lieabilityQAs"] is not None:
                sample["lieabilityQAs"]  = repl["lieabilityQAs"]
            num_replaced += 1

    print(f"已成功替换 {num_replaced} 条记录。")

    # 5. 写回
    save_json(target_data, TARGET_PATH)
    print(f"已保存更新后的文件 -> {TARGET_PATH}")

if __name__ == "__main__":
    main()

已备份目标文件 -> /Users/liuyiwei/python/tactful-tom/dataset/1. final_set/Tactful_conv_set_4.json.bak
已成功替换 25 条记录。
已保存更新后的文件 -> /Users/liuyiwei/python/tactful-tom/dataset/1. final_set/Tactful_conv_set_4.json


In [ ]:
# 两个json之间替换某一个问题

# import json

# def replace_data_by_set_id(
#     path_original="/Users/liuyiwei/python/tactful-tom/dataset/Tactful_conv_set_0.json",
#     path_replacement="/Users/liuyiwei/python/tactful-tom/dataset/Tactful_conv_set_0-1.json",
#     path_output="/Users/liuyiwei/python/tactful-tom/dataset/Tactful_conv_set_0_updated.json"
# ):
#     """
#     将 Tactful_conv_set_0-1.json 中的
#     1. infoAccessibilityQA_list
#     2. lieabilityQAs
#     3. justificationQAs (或 justification_question 等自定义字段)
#     通过 set_id 匹配，批量替换到 Tactful_conv_set_0.json 对应条目中
#     """

#     # 1. 读取原始数据
#     with open(path_original, "r", encoding="utf-8") as f:
#         data_original = json.load(f)

#     # 2. 读取要替换的文件数据
#     with open(path_replacement, "r", encoding="utf-8") as f:
#         data_replacement = json.load(f)

#     # 3. 分别建立三个映射字典
#     #    {set_id: infoAccessibilityQA_list}、{set_id: lieabilityQAs}、{set_id: justificationQAs}
#     infoAccessibility_map = {}
#     lieability_map = {}
#     justification_map = {}  # 用于存储 justificationQAs

#     for item in data_replacement:
#         set_id = item.get("set_id")
#         if set_id is not None:
#             # 如果有 infoAccessibilityQA_list，就放到 infoAccessibility_map
#             if "infoAccessibilityQA_list" in item:
#                 infoAccessibility_map[set_id] = item["infoAccessibilityQA_list"]
#             # 如果有 lieabilityQAs，就放到 lieability_map
#             if "lieabilityQAs" in item:
#                 lieability_map[set_id] = item["lieabilityQAs"]
#             # 如果有 justificationQAs，就放到 justification_map
#             # (若你的字段名不同，如 "justification_question"，请对应修改)
#             if "justificationQAs" in item:
#                 justification_map[set_id] = item["justificationQAs"]

#     # 4. 遍历原始数据，根据 set_id 替换对应字段
#     for item in data_original:
#         set_id = item.get("set_id")
#         if set_id in infoAccessibility_map:
#             item["infoAccessibilityQA_list"] = infoAccessibility_map[set_id]
#         if set_id in lieability_map:
#             item["lieabilityQAs"] = lieability_map[set_id]
#         if set_id in justification_map:
#             item["justificationQAs"] = justification_map[set_id]

#     # 5. 将更新后的数据写入新的 JSON 文件
#     with open(path_output, "w", encoding="utf-8") as f:
#         json.dump(data_original, f, ensure_ascii=False, indent=2)

# if __name__ == "__main__":
#     replace_data_by_set_id()
#     print("infoAccessibilityQA_list & lieabilityQAs & justificationQAs 替换完成！")

#### humanperformance问卷

In [35]:
import json
from IPython.display import display, Markdown

# ---------------------------
# 把数据读进来
# ---------------------------
output_filepath = "/Users/liuyiwei/python/tactful-tom/dataset/final_set/Tactful_conv_set_2.json"
with open(output_filepath, "r", encoding="utf-8") as f:
    data = json.load(f)


# ---------------------------
# 主函数
# ---------------------------
def show_set_markdown(data, set_id):
    """
    在 Notebook 中以 Markdown 形式打印指定 set_id 的
    full_context 及各类 QA。
    """
    # 1. 找到对应的 sample
    sample = next((x for x in data if x.get("set_id") == set_id), None)
    if not sample:
        print(f"⚠️  set_id = {set_id} 未找到")
        return

    # 2. 先打印对话 full_context
    md_lines = [f"## Set ID: {set_id}",
                "### Full Context"]
    for line in sample["full_context"].split("\n"):
        if line.strip():                      # 跳过空行
            md_lines.append(f"{line}  ")      # 两个空格代表换行

    # 3. 需要依次打印的 QA 字段与标题
    qa_fields = [
        ("comprehensionQA",        "Comprehension QA"),
        ("justificationQA",        "Justification QA"),
        ("liedetectabilityQAs_list","Lie-detectability QAs"),
        ("lieabilityQAs",          "Lie-ability QAs"),
        ("infoAccessibilityQA_list","Info-accessibility QAs"),
        ("answerabilityQA_list",   "Answerability QAs"),
        ("beliefQAs",              "Belief QAs")
    ]

    # 4. 逐个字段打印
    for key, nice_name in qa_fields:
        qa_list = sample.get(key, [])
        if not qa_list:            # 没有该字段就跳过
            continue

        md_lines.append("\n---")
        md_lines.append(f"### {nice_name}")

        for i, qa in enumerate(qa_list, 1):
            # 问题
            md_lines.append(f"**Q{i}. {qa.get('question','')}**  ")

            # 正确答案
            if "correct_answer" in qa:
                # md_lines.append(f"- ✅ Correct answer: {qa['correct_answer']}  ")
                md_lines.append(f"{qa['correct_answer']}  ")        # 注意行尾两个空格
            # 错误答案（有些字段叫 wrong_answer，有些叫 wrong_answers… 做个兜底）
            # # wrong_key = "wrong_answer" if "wrong_answer" in qa else "wrong_answers" if "wrong_answers" in qa else None
            # if wrong_key and qa[wrong_key]:
            #     for j, w in enumerate(qa[wrong_key], 1):
            #         # md_lines.append(f"- ❌ Wrong answer {j}: {w}  ")
            wrong_key = next((k for k in ("wrong_answer", "wrong_answers") if k in qa), None)
            if wrong_key and qa[wrong_key]:
                for w in qa[wrong_key]:
                    md_lines.append(f"{w}  ")                      # 同样补两个空格

            # 其余字段(如 question_type、q_id 等)想要展示的话也可以加
            extra_keys = [k for k in qa.keys() if k not in {
                "question", "correct_answer", "wrong_answer", "wrong_answers"}]
            for k in extra_keys:
                md_lines.append(f"- *{k}*: {qa[k]}  ")

            md_lines.append("")    # 额外空行，Markdown 看着更舒服

    # 5. 显示出来
    display(Markdown("\n".join(md_lines)))


# ---------------------------
# 调用示例
# ---------------------------
# type0
# show_set_markdown(data, set_id="0-1-0-0")   # 把 set_id 换成你想看的
# show_set_markdown(data, set_id="0-1-0-1")   # 把 set_id 换成你想看的
# show_set_markdown(data, set_id="0-2-0-2")   # 把 set_id 换成你想看的

# type1
# show_set_markdown(data, set_id="1-6-0-1")   # 把 set_id 换成你想看的
# show_set_markdown(data, set_id="1-7-0-1")   # 把 set_id 换成你想看的
# show_set_markdown(data, set_id="1-8-0-1")   # 把 set_id 换成你想看的

# type2
# show_set_markdown(data, set_id="2-10-0-0")   # 把 set_id 换成你想看的
show_set_markdown(data, set_id="2-10-2-2")   # 把 set_id 换成你想看的
# show_set_markdown(data, set_id="2-10-3-1")   # 把 set_id 换成你想看的

# type3
# 3-11-0-1
# 3-11-0-2
# 2-10-2-2


# type4
# 4-12-0-1
# 4-12-0-2
# 4-12-1-1

⚠️  set_id = 2-10-2-2 未找到


In [2]:
import json

# --- 配置区域 ---
# 这是所有需要删除的 q_id 的完整列表
ALL_Q_IDS_TO_DELETE_IN_RESULTS = {
    "2-10-0-0-info_access_list-0", "2-10-0-1-info_access_list-0",
    "2-10-1-0-info_access_list-0", "2-10-1-1-info_access_list-0",
    "2-10-2-1-info_access_list-0", "2-10-3-0-info_access_list-0",
    "2-10-3-1-info_access_list-0", "2-10-2-0-info_access_list-0",
    "2-10-2-2-info_access_list-0", "2-10-4-0-info_access_list-0",
    "2-10-4-1-info_access_list-0", "2-10-5-0-info_access_list-0",
    "2-10-5-1-info_access_list-0", "2-10-5-2-info_access_list-0",
    "2-10-0-0-info_access_binary-0", "2-10-0-1-info_access_binary-0",
    "2-10-1-0-info_access_binary-0", "2-10-1-1-info_access_binary-0",
    "2-10-2-1-info_access_binary-0", "2-10-3-0-info_access_binary-0",
    "2-10-3-1-info_access_binary-0", "2-10-0-0-lieability-0",
    "2-10-0-1-lieability-0", "2-10-1-0-lieability-0",
    "2-10-1-1-lieability-0", "2-10-2-1-lieability-0",
    "2-10-3-0-lieability-0", "2-10-3-1-lieability-0",
    "2-10-2-2-lieability-0", "2-10-4-0-lieability-0",
    "2-10-4-1-lieability-0", "2-10-5-0-lieability-0",
    "2-10-5-1-lieability-0", "2-10-5-2-lieability-0",
    "2-10-0-0-lieability-1", "2-10-0-1-lieability-2",
    "2-10-1-0-lieability-1", "2-10-1-1-lieability-2",
    "2-10-2-1-lieability-2", "2-10-3-0-lieability-1",
    "2-10-3-1-lieability-2",
}

# 所有可能包含问答结果的键名
ALL_QA_KEYS = [
    "comprehensionQA", "justificationQA", "fact_reasonQA", "fact_truthQA",
    "beliefQAs", "infoAccessibilityQA_list", "infoAccessibilityQAs_binary",
    "answerabilityQA_list", "answerabilityQAs_binary",
    "liedetectabilityQAs_list", "liedetectabilityQAs_binary", "lieabilityQAs",
]


# --- 主脚本 ---

def clean_results_file_robust(input_filepath, output_filepath):
    """
    加载结果JSON文件，根据q_id列表删除条目，能同时处理多种数据结构和ID键名。
    """
    try:
        with open(input_filepath, "r", encoding="utf-8") as f:
            data = json.load(f)
    except FileNotFoundError:
        print(f"错误: 输入文件 '{input_filepath}' 未找到。")
        return
    except json.JSONDecodeError:
        print(f"错误: 文件 '{input_filepath}' 不是有效的JSON格式。")
        return

    # 遍历每个对话的结果集
    for result_set in data:
        # 遍历所有可能的问答列表键
        for key in ALL_QA_KEYS:
            # 检查键是否存在且其值为列表
            if key not in result_set or not isinstance(result_set[key], list):
                continue

            cleaned_list = []
            original_list = result_set[key]

            # 遍历外层列表的每个元素
            for element in original_list:
                # 情况一：元素是子列表，对应 [[{...}]] 结构
                if isinstance(element, list):
                    filtered_sublist = [
                        qa for qa in element
                        # 核心修正：同时检查 "question_id" 和 "q_id"
                        if qa.get("question_id") not in ALL_Q_IDS_TO_DELETE_IN_RESULTS and \
                           qa.get("q_id") not in ALL_Q_IDS_TO_DELETE_IN_RESULTS
                    ]
                    # 如果子列表过滤后还有内容，则保留
                    if filtered_sublist:
                        cleaned_list.append(filtered_sublist)
                
                # 情况二：元素是字典，对应 [{...}] 结构
                elif isinstance(element, dict):
                    # 核心修正：同时检查 "question_id" 和 "q_id"
                    q_id = element.get("question_id") or element.get("q_id")
                    if q_id not in ALL_Q_IDS_TO_DELETE_IN_RESULTS:
                        cleaned_list.append(element)
            
            # 用清理后的列表替换原始列表
            result_set[key] = cleaned_list

    # 将最终清理后的数据写入新文件
    with open(output_filepath, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4, ensure_ascii=False)

    print(f"结果清理完成！已将更正后的数据保存到 '{output_filepath}'。")


# --- 执行区域 ---

# if __name__ == "__main__":
#     # 请确保这里的路径和文件名是正确的
#     input_file = "/Users/liuyiwei/tactful-tom-main copy/results/clean/QwQ-32B-2 copy.json"
#     output_file = "/Users/liuyiwei/tactful-tom-main copy/results/clean/QwQ-32B-2.json"

#     clean_results_file_robust(input_file, output_file)
import os

if __name__ == "__main__":
    # 获取clean文件夹中所有的文件
    clean_dir = "/Users/liuyiwei/tactful-tom-main/results/clean/"
    all_files_in_dir = os.listdir(clean_dir)
    
    # 筛选出所有以-2.json结尾的文件
    type2_files = [name for name in all_files_in_dir 
                   if name.endswith('-2.json') and not name.startswith('.')]
    
    print(f"找到 {len(type2_files)} 个 -2.json 文件:")
    for file in type2_files:
        print(f"  - {file}")
    
    # 批量处理所有-2.json文件
    for file_name in type2_files:
        input_file = os.path.join(clean_dir, file_name)
        output_file = input_file  # 直接覆盖原文件
        
        print(f"\n正在处理: {file_name}")
        try:
            clean_results_file_robust(input_file, output_file)
            print(f"✓ 完成: {file_name}")
        except Exception as e:
            print(f"✗ 错误: {file_name} - {e}")
    
    print(f"\n批量处理完成！共处理了 {len(type2_files)} 个文件。")

找到 12 个 -2.json 文件:
  - gpt-4o-2024-08-06-2.json
  - DeepSeek-V3-0324-cot-2.json
  - Llama-3.3-70B-Instruct-cot-2.json
  - QwQ-32B-2.json
  - o3-mini-2025-01-31-2.json
  - o1-2024-12-17-2.json
  - Qwen2.5-72B-Instruct-cot-2.json
  - gpt-4o-2024-08-06-cot-2.json
  - DeepSeek-R1-Turbo-2.json
  - Qwen2.5-72B-Instruct-2.json
  - DeepSeek-V3-0324-2.json
  - Llama-3.3-70B-Instruct-2.json

正在处理: gpt-4o-2024-08-06-2.json
结果清理完成！已将更正后的数据保存到 '/Users/liuyiwei/tactful-tom-main/results/clean/gpt-4o-2024-08-06-2.json'。
✓ 完成: gpt-4o-2024-08-06-2.json

正在处理: DeepSeek-V3-0324-cot-2.json
结果清理完成！已将更正后的数据保存到 '/Users/liuyiwei/tactful-tom-main/results/clean/DeepSeek-V3-0324-cot-2.json'。
✓ 完成: DeepSeek-V3-0324-cot-2.json

正在处理: Llama-3.3-70B-Instruct-cot-2.json
结果清理完成！已将更正后的数据保存到 '/Users/liuyiwei/tactful-tom-main/results/clean/Llama-3.3-70B-Instruct-cot-2.json'。
✓ 完成: Llama-3.3-70B-Instruct-cot-2.json

正在处理: QwQ-32B-2.json
结果清理完成！已将更正后的数据保存到 '/Users/liuyiwei/tactful-tom-main/results/clean/QwQ-32B-2.json'。
✓ 

In [1]:
import json

# --- 配置区域 ---
# 这是你想要查找和计数的 q_id 完整列表
Q_IDS_TO_FIND = {
    "2-10-0-0-info_access_list-0", "2-10-0-1-info_access_list-0",
    "2-10-1-0-info_access_list-0", "2-10-1-1-info_access_list-0",
    "2-10-2-1-info_access_list-0", "2-10-3-0-info_access_list-0",
    "2-10-3-1-info_access_list-0", "2-10-2-0-info_access_list-0",
    "2-10-2-2-info_access_list-0", "2-10-4-0-info_access_list-0",
    "2-10-4-1-info_access_list-0", "2-10-5-0-info_access_list-0",
    "2-10-5-1-info_access_list-0", "2-10-5-2-info_access_list-0",
    "2-10-0-0-info_access_binary-0", "2-10-0-1-info_access_binary-0",
    "2-10-1-0-info_access_binary-0", "2-10-1-1-info_access_binary-0",
    "2-10-2-1-info_access_binary-0", "2-10-3-0-info_access_binary-0",
    "2-10-3-1-info_access_binary-0", "2-10-0-0-lieability-0",
    "2-10-0-1-lieability-0", "2-10-1-0-lieability-0",
    "2-10-1-1-lieability-0", "2-10-2-1-lieability-0",
    "2-10-3-0-lieability-0", "2-10-3-1-lieability-0",
    "2-10-2-2-lieability-0", "2-10-4-0-lieability-0",
    "2-10-4-1-lieability-0", "2-10-5-0-lieability-0",
    "2-10-5-1-lieability-0", "2-10-5-2-lieability-0",
    "2-10-0-0-lieability-1", "2-10-0-1-lieability-2",
    "2-10-1-0-lieability-1", "2-10-1-1-lieability-2",
    "2-10-2-1-lieability-2", "2-10-3-0-lieability-1",
    "2-10-3-1-lieability-2",
}

# 所有可能包含问答结果的键名
ALL_QA_KEYS = [
    "comprehensionQA", "justificationQA", "fact_reasonQA", "fact_truthQA",
    "beliefQAs", "infoAccessibilityQA_list", "infoAccessibilityQAs_binary",
    "answerabilityQA_list", "answerabilityQAs_binary",
    "liedetectabilityQAs_list", "liedetectabilityQAs_binary", "lieabilityQAs",
]


# --- 统计函数 ---

def count_qids_in_file(filepath, qids_to_find_set):
    """
    加载一个JSON文件，统计其中包含了多少个在指定集合中的 q_id，并打印结果。
    """
    try:
        with open(filepath, "r", encoding="utf-8") as f:
            data = json.load(f)
    except FileNotFoundError:
        print(f"错误: 文件 '{filepath}' 未找到。")
        return
    except json.JSONDecodeError:
        print(f"错误: 文件 '{filepath}' 不是有效的JSON格式。")
        return

    # 使用集合(set)来存储找到的ID，可以自动处理重复项
    found_ids = set()
    id_key_in_results = "question_id"  # 你已确认文件中使用的键是这个

    # 遍历数据结构以查找ID
    for item in data:
        for key in ALL_QA_KEYS:
            if key in item and isinstance(item[key], list):
                for element in item[key]:
                    # 处理 [[{...}]] 结构
                    if isinstance(element, list):
                        for qa in element:
                            q_id = qa.get(id_key_in_results)
                            if q_id in qids_to_find_set:
                                found_ids.add(q_id)
                    # 处理 [{...}] 结构
                    elif isinstance(element, dict):
                        q_id = element.get(id_key_in_results)
                        if q_id in qids_to_find_set:
                            found_ids.add(q_id)

    # 报告最终统计结果
    print(f"--- 统计报告 ---")
    print(f"文件: '{filepath}'")
    print(f"在你的删除列表中总共有 {len(qids_to_find_set)} 个 q_id。")
    print(f"在文件中实际找到了其中 {len(found_ids)} 个。")
    
    # (可选) 找出哪些ID没有在文件中找到
    not_found_ids = qids_to_find_set - found_ids
    if not_found_ids:
        print(f"\n有 {len(not_found_ids)} 个来自删除列表的 q_id 未在文件中找到:")
        for qid in sorted(list(not_found_ids)):
            print(f"  - {qid}")


# --- 执行区域 ---

if __name__ == "__main__":
    # 设置要检查的原始文件名
    input_file_to_check = "/Users/liuyiwei/tactful-tom-main/results/clean/QwQ-32B-2.json"
    
    # 调用统计函数
    count_qids_in_file(input_file_to_check, Q_IDS_TO_FIND)

--- 统计报告 ---
文件: '/Users/liuyiwei/tactful-tom-main/results/clean/QwQ-32B-2.json'
在你的删除列表中总共有 41 个 q_id。
在文件中实际找到了其中 0 个。

有 41 个来自删除列表的 q_id 未在文件中找到:
  - 2-10-0-0-info_access_binary-0
  - 2-10-0-0-info_access_list-0
  - 2-10-0-0-lieability-0
  - 2-10-0-0-lieability-1
  - 2-10-0-1-info_access_binary-0
  - 2-10-0-1-info_access_list-0
  - 2-10-0-1-lieability-0
  - 2-10-0-1-lieability-2
  - 2-10-1-0-info_access_binary-0
  - 2-10-1-0-info_access_list-0
  - 2-10-1-0-lieability-0
  - 2-10-1-0-lieability-1
  - 2-10-1-1-info_access_binary-0
  - 2-10-1-1-info_access_list-0
  - 2-10-1-1-lieability-0
  - 2-10-1-1-lieability-2
  - 2-10-2-0-info_access_list-0
  - 2-10-2-1-info_access_binary-0
  - 2-10-2-1-info_access_list-0
  - 2-10-2-1-lieability-0
  - 2-10-2-1-lieability-2
  - 2-10-2-2-info_access_list-0
  - 2-10-2-2-lieability-0
  - 2-10-3-0-info_access_binary-0
  - 2-10-3-0-info_access_list-0
  - 2-10-3-0-lieability-0
  - 2-10-3-0-lieability-1
  - 2-10-3-1-info_access_binary-0
  - 2-10-3-1-info_

In [20]:
import json

# --- 配置区域 ---
# (这部分保持不变)
DATASET_FILE_PATH = "/Users/liuyiwei/tactful-tom-main/dataset/final_set/Tactful_conv_set_2 copy.json"
RESULTS_FILE_PATH = "/Users/liuyiwei/tactful-tom-main/results/clean/QwQ-32B-2.json"

ALL_QA_KEYS = [
    "comprehensionQA", "justificationQA", "fact_reasonQA", "fact_truthQA",
    "beliefQAs", "infoAccessibilityQA_list", "infoAccessibilityQAs_binary",
    "answerabilityQA_list", "answerabilityQAs_binary",
    "liedetectabilityQAs_list", "liedetectabilityQAs_binary", "lieabilityQAs",
]

# --- 脚本函数 ---

def create_question_id_map(dataset_filepath):
    """
    从原始数据集中读取并创建一个字典，用于将 q_id 映射到其对应的 question 文本。
    """
    id_to_question_map = {}
    print(f"正在从 '{dataset_filepath}' 创建 ID-Question 映射...")
    try:
        with open(dataset_filepath, "r", encoding="utf-8") as f:
            data = json.load(f)
    except Exception as e:
        print(f"错误：读取数据集文件失败: {e}")
        return None

    for conversation in data:
        for key in ALL_QA_KEYS:
            if key in conversation and isinstance(conversation[key], list):
                for qa in conversation[key]:
                    q_id = qa.get("q_id")
                    question_text = qa.get("question")
                    if q_id and question_text:
                        id_to_question_map[q_id] = question_text
    
    print(f"创建映射成功！共找到 {len(id_to_question_map)} 个独特的 ID-Question 对。")
    return id_to_question_map

def verify_results_alignment_corrected(results_filepath, id_to_question_map):
    """
    (已修正) 检查结果文件，验证ID和Question文本是否与原始数据集完全对齐。
    """
    if not id_to_question_map:
        print("错误: ID-问题 映射为空，无法进行验证。")
        return

    print(f"\n正在验证文件 '{results_filepath}'...")
    try:
        with open(results_filepath, "r", encoding="utf-8") as f:
            results_data = json.load(f)
    except Exception as e:
        print(f"错误: 读取结果文件失败: {e}")
        return

    text_mismatches = []
    ghost_ids = []  # 用于存储那些不该存在的 "幽灵" ID
    total_results_checked = 0
    id_key_in_results = "question_id"

    for result_set in results_data:
        for key in ALL_QA_KEYS:
            if key in result_set and isinstance(result_set[key], list):
                for element in result_set[key]:
                    qa_list = element if isinstance(element, list) else [element]
                    
                    for qa_result in qa_list:
                        total_results_checked += 1
                        result_id = qa_result.get(id_key_in_results)
                        
                        if not result_id:
                            continue

                        # --- 核心逻辑修正 ---
                        # 1. 检查这个ID是否存在于原始数据集中
                        if result_id not in id_to_question_map:
                            ghost_ids.append(qa_result)
                        else:
                            # 2. 如果ID存在，再检查Question文本是否匹配
                            expected_question = id_to_question_map[result_id]
                            actual_question = qa_result.get("question")
                            if actual_question != expected_question:
                                text_mismatches.append({
                                    "id": result_id,
                                    "expected": expected_question,
                                    "actual": actual_question
                                })

    # --- 打印更详细的报告 ---
    print("\n--- 检查报告 ---")
    print(f"共检查了 {total_results_checked} 条结果。")
    
    is_successful = True

    if ghost_ids:
        is_successful = False
        print(f"\n❌ 严重错误: 在结果文件中发现了 {len(ghost_ids)} 个不存在于原始数据集的 '幽灵ID'！")
        for qa_result in ghost_ids:
            # 截取一部分问题文本方便查看
            question_snippet = (qa_result.get('question') or "N/A")[:70] + "..."
            print(f"  - ID: {qa_result.get(id_key_in_results)}, Question: \"{question_snippet}\"")
    
    if text_mismatches:
        is_successful = False
        print(f"\n❌ 错误: 发现了 {len(text_mismatches)} 个 'question' 文本不匹配的条目:")
        for mismatch in text_mismatches:
            print(f"\n  - ID: {mismatch['id']}")
            print(f"    - 期望的 Question: {mismatch['expected']}")
            print(f"    - 实际的 Question: {mismatch['actual']}")
    
    if is_successful:
        print("\n✅ 检查成功! 所有结果都与原始数据集完全匹配。")

# --- 执行区域 ---

if __name__ == "__main__":
    question_map = create_question_id_map(DATASET_FILE_PATH)
    if question_map:
        verify_results_alignment_corrected(RESULTS_FILE_PATH, question_map)

正在从 '/Users/liuyiwei/tactful-tom-main/dataset/final_set/Tactful_conv_set_2 copy.json' 创建 ID-Question 映射...
创建映射成功！共找到 568 个独特的 ID-Question 对。

正在验证文件 '/Users/liuyiwei/tactful-tom-main/results/clean/QwQ-32B-2.json'...

--- 检查报告 ---
共检查了 463 条结果。

❌ 严重错误: 在结果文件中发现了 326 个不存在于原始数据集的 '幽灵ID'！
  - ID: 2-11-0-0-comprehension-0, Question: "Was what Rona and Mimi told Camdyn, 'Rona and Mimi tell Camdyn that Ca..."
  - ID: 2-11-0-0-comprehension-0, Question: "Was what Rona and Mimi told Camdyn, 'Rona and Mimi tell Camdyn that Ca..."
  - ID: 2-11-0-0-fact_reason-0, Question: "Who discussed 'Camdyn will be heartbroken if hearing that her grandma ..."
  - ID: 2-11-0-0-fact_reason-0, Question: "Who discussed 'Camdyn will be heartbroken if hearing that her grandma ..."
  - ID: 2-11-0-0-belief-0, Question: "What does Rona believe about 'Who discussed 'Camdyn will be heartbroke..."
  - ID: 2-11-0-0-belief-0, Question: "What does Rona believe about 'Who discussed 'Camdyn will be heartbroke..."
  - ID: 2-